In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession, Row
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number, regexp_replace, sum, when, count, round as sparkrnd, max as sparkmax, min as sparkmin, year, avg as sparkavg, stddev as sparkstd
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import regexp_replace
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType


In [2]:
spark = SparkSession.builder.getOrCreate()

24/06/24 13:18:17 WARN Utils: Your hostname, dsbda-vm resolves to a loopback address: 127.0.1.1; using 192.168.64.2 instead (on interface enp0s1)
24/06/24 13:18:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/24 13:18:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
file_path = "/home/ubuntu/DATASET/listings_details.csv"
file_path_out = "/home/ubuntu/DATASET/listings_details_select"

df = spark.read.csv(file_path, header=True, inferSchema=True, sep=",", quote= '"', escape= '"', multiLine=True, mode="DROPMALFORMED")
df_cleaned = df.withColumn("price", regexp_replace(col("price"), r"[\$,]", "").cast("double"))
for col_name in df_cleaned.columns:
        df_cleaned= df_cleaned.withColumn(col_name, regexp_replace(col(col_name), "[^a-zA-Z0-9 .]", ""))
        #df_cleaned = df_cleaned.withColumn(col_name, regexp_replace(col(col_name), "[\n\t\r]", ""))
df_filtered = df_cleaned.filter( col("id").cast(IntegerType()).isNotNull()
)

#df_filtered.printSchema()
df_filtered = df_filtered.select("id", "name", "host_name", "neighbourhood_cleansed", "latitude", "longitude", 
                                 "property_type", "room_type", "price", "review_scores_rating", "review_scores_accuracy", 
                                 "review_scores_cleanliness", "review_scores_checkin", "review_scores_communication", 
                                 "review_scores_location", "review_scores_value", "reviews_per_month")
df_filtered.show()

df_filtered.write.csv(file_path_out, header=False)

# The following lines generate an error because it says that no namenode is running even if
#I run the command jps and I can see the running namenodes

#hdfs_path = "hdfs://localhost:54310/user/ubuntu/ciao/reviews_details_cleaned.csv"
#df_filtered.write.csv(hdfs_path, header=True, mode="overwrite")


+-----+--------------------+---------+----------------------+------------------+------------------+-----------------+--------------+-----+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+-----------------+
|   id|                name|host_name|neighbourhood_cleansed|          latitude|         longitude|    property_type|     room_type|price|review_scores_rating|review_scores_accuracy|review_scores_cleanliness|review_scores_checkin|review_scores_communication|review_scores_location|review_scores_value|reviews_per_month|
+-----+--------------------+---------+----------------------+------------------+------------------+-----------------+--------------+-----+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+-----------------+
| 2818|Quiet Garden View...|   Daniel|  

In [4]:
schema = StructType([
    StructField("listing_id", StringType(), True),
    StructField("id", StringType(), True),
    StructField("date", StringType(), True),
    StructField("reviewer_id", StringType(), True),
    StructField("reviewer_name", StringType(), True),
    StructField("comments", StringType(), True)
])

file_path = "/home/ubuntu/DATASET/reviews_details.csv"
file_path_out = "/home/ubuntu/DATASET/reviews_details_select"
df = spark.read.csv(file_path, header=True, schema=schema, sep=",", quote= '"', escape= '"', multiLine=True, mode="DROPMALFORMED")
for col_name in df.columns:
        if col_name == "date":
            df= df.withColumn(col_name, regexp_replace(col(col_name), "[^0-9 -]", ""))
        else:
            df= df.withColumn(col_name, regexp_replace(col(col_name), "[^a-zA-Z0-9 .]", ""))
df = df.filter(col("listing_id").cast("int").isNotNull())

df.printSchema()
#I have to limit the output because on spark it's costly!
df.show(20)

df.select("listing_id", "date", "comments").show()
df = df.select("listing_id", "date", "comments")

df.write.csv(file_path_out, header=False)

# This generates an error because it says that no namenode is running even if
#I run the command jps and I can see the running namenodes

#hdfs_path = "hdfs://localhost:54310/user/ubuntu/ciao/reviews_details_cleaned.csv"
#df.write.csv(hdfs_path, header=True, mode="overwrite")

root
 |-- listing_id: string (nullable = true)
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- reviewer_id: string (nullable = true)
 |-- reviewer_name: string (nullable = true)
 |-- comments: string (nullable = true)

+----------+-----+----------+-----------+-------------+--------------------+
|listing_id|   id|      date|reviewer_id|reviewer_name|            comments|
+----------+-----+----------+-----------+-------------+--------------------+
|      2818| 1191|2009-03-30|      10952|          Lam|Daniel is really ...|
|      2818| 1771|2009-04-24|      12798|        Alice|Daniel is the mos...|
|      2818| 1989|2009-05-03|      11869|      Natalja|We had such a gre...|
|      2818| 2797|2009-05-18|      14064|      Enrique|Very professional...|
|      2818| 3151|2009-05-25|      17977|      Sherwin|Daniel is highly ...|
|      2818| 4748|2009-06-29|      20192|          Jie|Daniel was a grea...|
|      2818| 5202|2009-07-07|      23055|      Vanessa|Daniel

24/06/24 13:19:17 WARN DFSClient: DataStreamer Exception                        
org.apache.hadoop.ipc.RemoteException(java.io.IOException): File /user/ubuntu/ciao/reviews_details_cleaned.csv/_temporary/0/_temporary/attempt_202406241319177049768596050772336_0005_m_000000_5/part-00000-d26b57ad-6e6d-4496-a815-5ee69019690b-c000.csv could only be replicated to 0 nodes instead of minReplication (=1).  There are 0 datanode(s) running and no node(s) are excluded in this operation.
	at org.apache.hadoop.hdfs.server.blockmanagement.BlockManager.chooseTarget4NewBlock(BlockManager.java:1832)
	at org.apache.hadoop.hdfs.server.namenode.FSDirWriteFileOp.chooseTargetForNewBlock(FSDirWriteFileOp.java:265)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.getAdditionalBlock(FSNamesystem.java:2591)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.addBlock(NameNodeRpcServer.java:880)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.addBlock(ClientNameno

Py4JJavaError: An error occurred while calling o129.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:496)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:251)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:110)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$5(SQLExecution.scala:103)
	at org.ap

24/06/24 13:57:10 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 354548 ms exceeds timeout 120000 ms
24/06/24 13:57:10 WARN SparkContext: Killing executors is not supported by current scheduler.


In [10]:
spark.stop()